In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from collections import Counter

# Loads each whole scenario as a sheet (For Later)

In [68]:
start_folder = 'AllCSVs'
csvs_scen = glob.glob(r'{}\*.csv'.format(start_folder))
df_scen = []

for c in csvs_scen:
    df = pd.read_csv(c)
    df_scen.append(df)
print(len(df_scen))
# df_list[0]

223


# Loads and seperates each car into its own sheet

In [83]:
csvs = glob.glob(r'{}\*.csv'.format(start_folder))
df_list = []

for c in csvs:
    df = pd.read_csv(c)
    for i in np.unique(df.sender):
        df_list.append(df[df.sender.isin([i])].copy())
    #     df[df.ID.isin([subs[0]])&df.Vigil.isin(vigils[0])].copy()
print(len(df_list))
# df_list[0]

47539


In [71]:
df_list[1000]

,type,time,sender,attackerType,messageID,pos0,pos1,pos2,spd_noise0,spd_noise1,spd_noise2,pos_noise0,pos_noise1,pos_noise2,spd0,spd1,spd2
55,4,18001.941857,187,0,2888,5307.437889,5552.622747,1.895,0.0,0.0,0.0,0.0,0.0,0.0,1.398388,-1.204302,0.0
88,4,18002.941857,187,0,4671,5308.792560,5550.107778,1.895,0.0,0.0,0.0,0.0,0.0,0.0,2.213276,-2.886655,0.0
121,4,18003.941857,187,0,6519,5311.002746,5546.004531,1.895,0.0,0.0,0.0,0.0,0.0,0.0,2.722779,-4.762929,0.0
156,4,18004.941857,187,0,8554,5314.142093,5540.176279,1.895,0.0,0.0,0.0,0.0,0.0,0.0,3.571401,-6.630367,0.0
193,4,18005.941857,187,0,10856,5319.673865,5534.002615,1.895,0.0,0.0,0.0,0.0,0.0,0.0,5.817960,-7.481643,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3303,4,18082.941857,187,0,216013,5784.605373,5206.749164,1.895,0.0,0.0,0.0,0.0,0.0,0.0,11.877898,-5.910096,0.0
3342,4,18083.941857,187,0,218097,5796.438791,5200.861201,1.895,0.0,0.0,0.0,0.0,0.0,0.0,11.787682,-5.865207,0.0
3382,4,18084.941857,187,0,220343,5808.309060,5194.954901,1.895,0.0,0.0,0.0,0.0,0.0,0.0,11.863562,-5.902962,0.0
3421,4,18085.941857,187,0,222474,5820.142699,5189.066827,1.895,0.0,0.0,0.0,0.0,0.0,0.0,11.794506,-5.868602,0.0


# Note: you would find and organize the cars that are the closest right here and more the dataframe with that info now (Ex: for a given row with Car7 look N rows up and pick different carX with closest POS (Not Sped))

In [72]:
df_attacker = []
df_normal = []
window_size = 11
overlap_size = 10

for j in df_list:
    if j.attackerType.any() == 0:
        j = j.iloc[:,[1,2,5,6,14,15]]
        for start_idx in range(0, len(j)-window_size, window_size - overlap_size):
            end_idx = start_idx + window_size
            df_chunk = j.iloc[start_idx:end_idx]
#             display(df_chunk)
            df_normal.append(df_chunk)
        
        
    elif j.attackerType.any() == 1:
        j = j.iloc[:,[5,6,14,15]]
        df_attacker.append(j)

# df_normal = np.asarray(df_normal)
# df_attacker = np.asarray(df_attacker)

df_normal[0]

,time,sender,pos0,pos1,spd0,spd1
1,10800.437587,7,3609.398494,5446.803264,-3.534058,30.622234
8,10801.437587,7,3605.873370,5477.348089,-3.533871,30.620620
14,10802.437587,7,3603.260815,5507.988690,-2.516747,30.722842
20,10803.437587,7,3600.748307,5538.659784,-2.513009,30.677207
26,10804.437587,7,3598.236829,5569.318316,-2.512296,30.668505
32,10805.437587,7,3595.724619,5599.985772,-2.510090,30.641585
39,10806.437587,7,3593.213558,5630.639206,-2.514574,30.696319
45,10807.437587,7,3591.439053,5661.354795,-1.284169,30.756852
52,10808.437587,7,3590.156264,5692.078590,-1.287250,30.830647
59,10809.437587,7,3588.873085,5722.811723,-1.281963,30.704012


In [74]:
len(df_normal)
df_normal[1196210]

,time,sender,pos0,pos1,spd0,spd1
18749,25286.417793,2659,3645.011025,5184.868257,-4.619255,29.612163
18960,25287.417793,2659,3640.400515,5214.424359,-4.612973,29.571890
19168,25288.417793,2659,3635.791493,5243.970927,-4.620604,29.620810
19377,25289.417793,2659,3631.186669,5273.490579,-4.607912,29.539451
19583,25290.417793,2659,3626.578078,5303.034379,-4.617844,29.603123
19795,25291.417793,2659,3622.572370,5332.653032,-3.427165,29.696016
20006,25292.417793,2659,3619.144681,5362.353589,-3.433634,29.752070
20220,25293.417793,2659,3615.715848,5392.064063,-3.434611,29.760538
20435,25294.417793,2659,3612.286712,5421.777163,-3.428854,29.710654
20651,25295.417793,2659,3608.858331,5451.483719,-3.433412,29.750145


# Next take df_normal and for each element in it put 10 rows in X and 1 row into Y (Convert to numpy) For Model 1

In [75]:
X = []
y = []
for i in df_normal:
    X.append(np.asarray(df_normal[0].iloc[0:10,2:]))
    y.append(np.asarray(df_normal[0].iloc[10,2:]))
X = np.asarray(X)
y = np.asarray(y)
np.save(r'{}\X.npy'.format(start_folder), X)
np.save(r'{}\y.npy'.format(start_folder), y)

In [81]:
X1 = np.load(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\X.npy'.format(start_folder))
X1.shape

(1196218, 10, 4)

In [82]:
X1[0]

array([[ 3.60939849e+03,  5.44680326e+03, -3.53405773e+00,
         3.06222342e+01],
       [ 3.60587337e+03,  5.47734809e+03, -3.53387139e+00,
         3.06206197e+01],
       [ 3.60326082e+03,  5.50798869e+03, -2.51674686e+00,
         3.07228422e+01],
       [ 3.60074831e+03,  5.53865978e+03, -2.51300856e+00,
         3.06772074e+01],
       [ 3.59823683e+03,  5.56931832e+03, -2.51229565e+00,
         3.06685046e+01],
       [ 3.59572462e+03,  5.59998577e+03, -2.51009046e+00,
         3.06415851e+01],
       [ 3.59321356e+03,  5.63063921e+03, -2.51457414e+00,
         3.06963190e+01],
       [ 3.59143905e+03,  5.66135479e+03, -1.28416894e+00,
         3.07568515e+01],
       [ 3.59015626e+03,  5.69207859e+03, -1.28725005e+00,
         3.08306467e+01],
       [ 3.58887309e+03,  5.72281172e+03, -1.28196275e+00,
         3.07040115e+01]])